<h1>Web Scraping</h1>

O processo consiste em abrir o site "https://www.nfl.com/", acessar a aba "Stats", validar o carregamento da página, após o carregamento selecionar a aba "Team Stats"e validar novamente a abertura da tabela. Após a validação, capturar os dados da tabela e guardar em um array.

<h2>Libs</h2>

In [1]:
# Lib Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import selenium.common.exceptions

# Other libs
import os
import os.path
import json
import autoit
import time

<h2>Função Main</h2>

In [2]:
def main():
    
    #_______________________________________ INICIA O CHROMEDRIVER ________________________________ #
    
    driver_chrome = abrir_chrome()
    
    #_______________________________________ ACESSA SITE NFL ______________________________________ #

    
    url_site = "https://www.nfl.com/"

    retorno_abre_site = abre_site_nfl(
        url = url_site,
        driver = driver_chrome
    )

    if(retorno_abre_site == 0):
        print("Sucesso ao abrir o site nfl")
        
    elif(retorno_abre_site == -1):
        print("Não foi possivel identificar a abertura do site.")
        return False
    
    else:
        print("Erro inesperado abre_site_nfl() " + str(retorno_abre_site))
        
        
    #_______________________________________ ACESSA STATUS TIME ______________________________________ #
    
    acessa_status_time_resultado = acessa_status_time(
        titulo_busca = "PASSING",              
        driver = driver_chrome              
    )
    
    if(acessa_status_time_resultado == 0):
        print("Pagina com dados dos times acessada com sucesso")
        
    elif(acessa_status_time_resultado == -1):
        print("Não foi possivel encontrar o campo stats para click")
        return False
    
    elif(acessa_status_time_resultado == -2):
        print("Não foi possivel encontrar o campo team stats para click")
        return False
    
    else:
        print("Erro inesperado acessa_status_time() " + str(acessa_status_time_resultado))
    
    #_______________________________________ EXTRAI DADOS ______________________________________ #
    
    extrai_dados_resultado = extrai_dados(
        name_table = "Team", 
        driver = driver_chrome
    )
    
    type_extrai_dados = type(extrai_dados_resultado)
    print(extrai_dados_resultado)
    
    if("list" in str(type_extrai_dados)):
        print("Dados extraidos com sucesso")
        
    elif(extrai_dados_resultado == -1):
        print("Não foi possivel localizar a tabela")
        return False
    
    elif(extrai_dados_resultado == -2):
        print("Não foi possivel encontrar o campo team stats para click")
        return False
    
    else:
        print("Erro inesperado extrai_dados() " + str(extrai_dados_resultado))
        
    #_______________________________________ ENCERRA O CHROMEDRIVER _____________________________ #
    
    driver_chrome.quit()
    
    
    return

<h2> Funções Principais</h2>

In [3]:
def abre_site_nfl(url,driver,*, timeout = 10): 
    """
    Inputs
        url                    # Url de entrada para acessar o cpf de cada cliente
    Outputs
        -1                     # Não foi possivel identificar a abertura do site.
        0                      # Site aberto com sucesso
    """
    # -- Variaveis Funcao -- #
    contador = 0

    print("[Função]--> acessa_site_nfl")
    
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    original_window = driver.current_window_handle
    assert len(driver.window_handles) == 1
    wait.until(EC.number_of_windows_to_be(1))
    for window_handle in driver.window_handles:

        if window_handle != original_window:
            print("Switch window")
            driver.switch_to.window(window_handle)
            
            break

    WaitReadyState(driver)
    try:
        autoit.win_activate("NFL.com - Official Site")
    except:
        print("Janela nfl não existe")
    
    while(contador < timeout):
        
        time.sleep(1)
            
        try:
            all_options = driver.find_elements_by_tag_name("span")
            
        except:
            continue
            
        for option in all_options:
            
            try:
                verifica_site = option.get_attribute('innerText')
                print(verifica_site)

            except:
                continue
            
            if("News" in verifica_site):
                print("Site aberto com sucesso")
                return 0
            
        contador += 1
        print(contador)
                
    print("Não foi possivel identificar a abertura do site")
    return -1


In [4]:
def acessa_status_time(titulo_busca,driver,*, timeout = 10):
    """
    Inputs
        url                    # Url de entrada para acessar o cpf de cada cliente
    Outputs
        -1                     # Não foi possivel encontrar o campo stats para click
        -2                     # Não foi possivel encontrar o campo team stats para click
    """
    # -- Variaveis Funcao -- #
    contador = 0
    flag_stats = False
    flag_team_stats = False
    
    print("[Função]--> acessa_status_time")
    
    # fecha o popup, se houver
    fecha_popup_resultado = fecha_popup(driver = driver)
    print(str(fecha_popup_resultado))
    
    while(contador < timeout):
        
        time.sleep(1)
                
        try:
            navigation = driver.find_elements_by_tag_name("a")
            
        except:
            continue
            
        for option_navigation in navigation:
            
            try:
                filtra_campo_stats = option_navigation.get_attribute('innerText')
                print(filtra_campo_stats)

            except:
                continue
            
            if("Stats" in filtra_campo_stats):
                print("Clicando no campo stats")
                flag_stats = True
                option_navigation.click()
                break
                
        if(flag_stats):
            break
            
        contador += 1
        
    if not(flag_stats):
        print("Não foi possivel encontrar o campo stats para click")
        return -1
    
    # Se não houver popup anteriormente, tentar fechar o popup, se houver
    if not(fecha_popup_resultado):
        print("Verificando popup novamente")
        fecha_popup(driver = driver)
    
    contador = 0
    while(contador < timeout):

        try:
            titulos = driver.find_elements_by_tag_name("h2")
            
        except:
            continue
            
        for titulo in titulos:
            
            try:
                txt_titulo = titulo.get_attribute('innerText')
                print(txt_titulo)

            except:
                continue
            
            if(titulo_busca in txt_titulo):
                print("Clicando no campo team stats")
                flag_team_stats = True
                driver.execute_script('document.getElementsByTagName("a")[32].click()')
                return 0
            
        contador += 1
        print(contador)
        
    if not(flag_team_stats):
        print("Não foi possivel encontrar o campo team stats para click")
        return -2
    

In [5]:
def extrai_dados(name_table,driver,*,timeout = 60):
    """
    Inputs
        url                    # Url de entrada para acessar o cpf de cada cliente
        name_table             # Nome da tabela
    Outputs
        -1                     # Não foi possivel localizar a tabela
        -2                     # Não foi possivel encontrar o campo team stats para click
        guarda_dados           # Retorna a lista com os dados
    """
    # -- Variaveis Funcao -- #
    verifica_table = ""
    length_table = ""
    dados_tr = ""
    contador = 0
    guarda_dados = []
    
    while(contador <= timeout):
        
        time.sleep(1)
        
        try:
            verifica_table = driver.execute_script('return document.getElementsByTagName("table")[0].children[0].innerText')
            print(verifica_table)
        
        except:
            continue
            
        if(name_table in verifica_table):
            print("Tabela ja esta carregada")
            break
            
        elif(contador >= timeout):
            print("Não foi possivel localizar a tabela")
            return -1
        
        contador += 1
        
    length_table = driver.execute_script(
        'return document.getElementsByTagName("tbody")[0].getElementsByTagName("tr").length'
    )
    
    contador = 0
    
    while(contador < length_table):
        
        dados_tr = driver.execute_script(
            'return document.getElementsByTagName("table")[0].getElementsByTagName("tr")["' + str(contador) + '"].innerText'
        )
        
        print(dados_tr)
        
        guarda_dados.append(dados_tr)
        
        contador += 1
        
    return guarda_dados

<h2>Funções auxiliares</h2>

In [6]:
def fecha_popup(*,driver):
    
    contador = 0
    
    while(contador <= 10):
        
        try:
            btn_allow = driver.execute_script('return document.getElementById("onesignal-slidedown-allow-button").innerText')

            if("Allow" in btn_allow):
                print("Clicando no popup")
                driver.execute_script('document.getElementById("onesignal-slidedown-allow-button").click()')
                return True

            else:
                print("Popup não existe")
        except:
            pass
        
        time.sleep(1)
        
        contador += 1
    
    return False


In [7]:
def abrir_chrome(maximize=True):
    
    """
    Função abrir_chrome:
    
    Entrada:
        
    Retornos: 
        driver                        # Sucesso ao abrir o chrome.
    """
    print("\n\n[Função]--> abrir_chrome")

    path_id_chrome = r"C:\Users\gamer\Desktop\Projetos\Python\web_scraping\chromedriver\chrome_id.json"
    path_driver = r"C:\Users\gamer\Desktop\Projetos\Python\web_scraping\chromedriver\chromedriver.exe"

    while True:

        session_id, remote_machine_url = False, False
        if os.path.isfile(path_id_chrome):
            with open(path_id_chrome, "r") as file:
                session_id_as_str = file.read()
                try:
                    remote_machine_url, session_id = session_id_as_str.split(" ")
                except:
                    pass
        
        # Precisa para manter a sessao do Selenium aberta durante os diferente NoseTests
        global driver

        if session_id:
            driver = attach_to_session(remote_machine_url, session_id)
            
            if driver:
                try:
                    for handle in driver.window_handles:
                        driver.switch_to.window(handle)
                        break
                    return driver
                except: 
                    print("Error attach_to_session")
                    
                    with open(path_id_chrome, "w") as file:
                        file.write("")
        
        options = Options()
        options.add_experimental_option("excludeSwitches", ["enable-automation"])

        driver = webdriver.Chrome(executable_path=path_driver, chrome_options=options)

        driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'})

        if maximize:
            driver.maximize_window()

        remote_machine_url = driver.command_executor._url
        session_id = driver.session_id
        session_id_as_str = str(remote_machine_url) + " " + str(session_id)

        with open(path_id_chrome, "w") as file:
            file.write(session_id_as_str)

        return driver
    


In [8]:
def WaitReadyState(driver, timeout = 60, textException = ""):
    indTimeout = 0
    elementFounded = False
    readyState = ""
    if textException == "":
        textException = driver.title
    while indTimeout <= timeout and elementFounded != True:
        try:
            readyState = driver.execute_script("return document.readyState")
            if readyState == "complete" :
                elementFounded = True
        except:
            time.sleep(1)
            indTimeout += 1
    if indTimeout > timeout :
        print("Timeout ao esperar carregamento do browser")
    return

In [9]:
def reset_chrome_session():
    path_id_chrome = r"C:\Users\gamer\Desktop\Projetos\Python\web_scraping\chromedriver\chrome_id.json"
    
    try:
        with open(path_id_chrome, "w") as f:
            json.dump("", f)
        return True
    except FileNotFoundError:
        return False
    

In [10]:
def attach_to_session(executor_url, session_id):
    try:
        driver = webdriver.Remote(command_executor=executor_url, desired_capabilities={})
        driver.close()
        driver.session_id = session_id
        driver.implicitly_wait(0) # seconds
        return driver
    except:
        return None

<h1>Execução</h1>

In [11]:
main()



[Função]--> abrir_chrome


C:\Users\gamer\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: use options instead of chrome_options


[Função]--> acessa_site_nfl
Skip to main content

        
          
            Open menu button
          
        
      

          
            Open menu button
          
        

            Open menu button
          

News
Site aberto com sucesso
Sucesso ao abrir o site nfl
[Função]--> acessa_status_time
Clicando no popup
True
Skip to main content

News
Scores
Schedule
Videos
Teams
Players
Stats
Clicando no campo stats
PASSING
Clicando no campo team stats
Pagina com dados dos times acessada com sucesso
Team	Att	Cmp	Cmp %	Yds/Att	Pass Yds	TD	INT	Rate	1st	1st%	20+	40+	Lng	Sck	SckY

Tabela ja esta carregada
Team	Att	Cmp	Cmp %	Yds/Att	Pass Yds	TD	INT	Rate	1st	1st%	20+	40+	Lng	Sck	SckY
49ers
	570	371	65.1	7.6	4320	25	17	90.1	217	38.1	55	10	76T	39	287
Bears
	614	402	65.5	6.4	3925	26	16	86.5	218	35.5	42	3	53	36	270
Bengals
	581	372	64	6.5	3793	19	11	85.7	201	34.6	35	5	72T	48	345
Bills
	596	410	68.8	8	4786	40	11	107.6	240	40.3	66	10	56T	27	166
Broncos
	556	317	57	6.6	3673	21	23	72.5

<h1>Testes Unitários</h1>

In [11]:
def test_abre_site_nfl():
    
    url_site = "https://www.nfl.com/"

    test_case = [
        {
            "test_case_name": "Caso de erro",
            "url": "https://www.google.com.br/",
            "expected_result": "-1" 
        },
        {
            "test_case_name": "Caso de sucesso",
            "url": url_site,
            "expected_result": "0"
        }        
    ]
    
    for case in test_case:
        
        driver_chrome = abrir_chrome()
        result = abre_site_nfl(
            url = case['url'],
            driver = driver_chrome
        )
        print("Resultado test_abre_site_nfl()")
        print("Retorno func = " + str(result))
        print("expected_result = " + str(case["expected_result"]))
        assert str(result) in str(case["expected_result"])
        print("test_abre_site_nfl(): Sucesso!\n\n")
        driver_chrome.quit()
        
def test_acessa_status_time():
    
    url_site = "https://www.nfl.com/"

    test_case = [
        {
            "test_case_name": "Caso de erro",
            "title": "CASO DE ERRO",
            "expected_result": "-2" 
        },
        {
            "test_case_name": "Caso de sucesso",
            "title": "PASSING",
            "expected_result": "0"
        }        
    ]
    
    for case in test_case:
        
        driver_chrome = abrir_chrome()
        abre_site_nfl_resultado = abre_site_nfl(
            url = url_site,
            driver = driver_chrome
        )
        
        result = acessa_status_time(
            titulo_busca = case['title'],              
            driver = driver_chrome              
        )
        print("Resultado test_acessa_status_time()")
        print("Retorno func = " + str(result))
        print("expected_result = " + str(case["expected_result"]))
        assert str(result) in str(case["expected_result"])
        print("test_acessa_status_time(): Sucesso!\n\n")
        driver_chrome.quit()
        
def test_extrai_dados_resultado():
    
    url_site = "https://www.nfl.com/"

    test_case = [
        {
            "test_case_name": "Caso de erro",
            "title": "CASO DE ERRO",
            "timeout": 2,
            "expected_result": "-1" 
        },
        {
            "test_case_name": "Caso de sucesso",
            "title": "Team",
            "expected_result": "list"
        }        
    ]
    
    for case in test_case:
        
        timeout = case.get("timeout", None) or 60
        
        driver_chrome = abrir_chrome()
        abre_site_nfl_resultado = abre_site_nfl(
            url = url_site,
            driver = driver_chrome
        )
        
        acessa_status_time_resultado = acessa_status_time(
            titulo_busca = "PASSING",              
            driver = driver_chrome              
        )
        
        result = extrai_dados(
            name_table = case['title'], 
            driver = driver_chrome,
            timeout = timeout
        )
        
        type_result = type(result)
        print("Resultado test_extrai_dados_resultado()")
        print("Retorno result = " + str(result))
        print("Retorno type_result = " + str(type_result))
        print("expected_result = " + str(case["expected_result"]))
        assert str(case["expected_result"]) in str(type_result) or str(result) in str(case["expected_result"])
        print("test_extrai_dados_resultado(): Sucesso!\n\n")
        driver_chrome.quit()
    
    
# ______________________EXECUÇÃO TESTES UNITÁRIOS______________________#

test_abre_site_nfl()
test_acessa_status_time()
test_extrai_dados_resultado()
        



[Função]--> abrir_chrome


C:\Users\gamer\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: use options instead of chrome_options


[Função]--> acessa_site_nfl
Skip to main content

        
          
            Open menu button
          
        
      

          
            Open menu button
          
        

            Open menu button
          

News
Site aberto com sucesso
[Função]--> acessa_status_time
Clicando no popup
True
Skip to main content

News
Scores
Schedule
Videos
Teams
Players
Stats
Clicando no campo stats
PASSING
Clicando no campo team stats
Team	Att	Cmp	Cmp %	Yds/Att	Pass Yds	TD	INT	Rate	1st	1st%	20+	40+	Lng	Sck	SckY

Team	Att	Cmp	Cmp %	Yds/Att	Pass Yds	TD	INT	Rate	1st	1st%	20+	40+	Lng	Sck	SckY

Team	Att	Cmp	Cmp %	Yds/Att	Pass Yds	TD	INT	Rate	1st	1st%	20+	40+	Lng	Sck	SckY

Não foi possivel localizar a tabela
Resultado test_extrai_dados_resultado()
Retorno result = -1
Retorno type_result = <class 'int'>
expected_result = -1
test_extrai_dados_resultado(): Sucesso!




[Função]--> abrir_chrome
[Função]--> acessa_site_nfl
Skip to main content

        
          
            Open menu button